# Target Description Language

In [ ]:
from keckODL.target import Target, TargetList

In [ ]:
# Example: fully specify target.  Coordinates in decimal degrees.
t1 = Target(name='M42', RA=82.5, Dec=-5, frame='icrs',
            PMRA=10, PMDec=-10, epoch=1975.5,
            rotmode='PA', PA=90,
            objecttype='science',
            RAOffset=8.4, DecOffset=-4.5,
            acquisition='guider: offset',
            mag={'V': 4.3, 'K': 6.7},
            comment='Star formation is cool',
            wrap='shortest',
           )
t1

In [ ]:
# Example: fully specify target.  Coordinates in hmsdms.
t2 = Target(name='M31', RA='00:42:44.33', Dec='41:16:07.50',
            rotmode='PA', PA=32,
            RAOffset=12.1, DecOffset=-1.9,
            objecttype='science',
            acquisition='guider: offset',
            mag={'V': 2.3, 'K': 8.7},
            comment='Galaxies are boring',
            wrap='north',
           )
t2

In [ ]:
# Example: Resolve target by name.
t3 = Target('NGC1333',
            rotmode='PA', PA=22.5,
            objecttype='science',
            acquisition='blind',
            comment='Low mass star formation is the coolest')
t3

In [ ]:
# Example: Resolve target by name, set vertical angle mode, set obstype
t4 = Target('Feige110',
            objecttype='flux standard',
            rotmode='vertical',
            acquisition='guider: bright',
            comment='I guess we need a standard')
t4

In [ ]:
# Build a target list, write as TDL TAML file and as starlist
tl = TargetList([t1, t2, t3, t4])
tl.write('targets.txt')
tl.write_starlist('starlist.txt')
tl

In [ ]:
# Read in a target list from the TDL YAML file
newtl = TargetList().read('targets.txt')
newtl

In [ ]:
from astropy.time import Time
tl.set_obstime(Time(2045.11, format='decimalyear'))
tl

# Observing Sequences

## KCWI Example

In [1]:
from keckODL.detector_config import KCWIblueDetectorConfig
from keckODL.instrument_config import KCWIblueConfig
from keckODL.offset import ABBA, Stare, StarSkyStar
from keckODL.sequence import SequenceElement, Sequence

In [2]:
kcwi_1800s = KCWIblueDetectorConfig(exptime=1800, binning='2x2')
kcwi_1800s.write('kcwi_1800s.txt')
print(kcwi_1800s)

1800.0s (10)


In [3]:
med_slicer_4800 = KCWIblueConfig(slicer='medium', grating='BH3', cwave=4800)
print(med_slicer_4800)

medium BH3 4800 A


In [4]:
skyoffset = StarSkyStar(dx=30, dy=30)
print(skyoffset)

StarSkyStar (30 30)


In [5]:
skyoffset

 dx(")| dy(")| dr(deg)|    name
------|------|--------|--------
  +0.0|  +0.0|    +0.0|    star
 +30.0| +30.0|    +0.0|     sky
  +0.0|  +0.0|    +0.0|    star

In [6]:
myseq = Sequence([SequenceElement(pattern=skyoffset,
                                  detconfig=kcwi_1800s,
                                  instconfig=med_slicer_4800,
                                  repeat=3),
                  ])
print(myseq)

Pattern            |DetectorConfig               |InstrumentConfig             |repeat
-------------------|-----------------------------|-----------------------------|------
StarSkyStar (30 30)|1800.0s (10)                 |medium BH3 4800 A            |     3


In [7]:
myseq.estimate_time()

Shutter Open Time: 16200 s (4.5 hrs)


{'shutter open time': 16200, 'wall clock time': 0}

## MOSFIRE Example

In [ ]:
from keckODL.detector_config import MOSFIREDetectorConfig
from keckODL.instrument_config import MOSFIREConfig
from keckODL.offset import ABBA, Long2pos
from keckODL.sequence import SequenceElement, Sequence

In [ ]:
# detector configs
mosfire_180s = MOSFIREDetectorConfig(exptime=180, readoutmode='MCDS16', coadds=1)
mosfire_180s.write('mosfire_180s.txt')
mosfire_120s = MOSFIREDetectorConfig(exptime=120, readoutmode='MCDS16', coadds=1)
mosfire_120s.write('mosfire_120s.txt')
print(mosfire_180s)
print(mosfire_120s)

In [ ]:
# instrument configs
mosfire_Y = MOSFIREConfig(mode='spectroscopy', filter='Y', mask='my_MOS_mask')
mosfire_J = MOSFIREConfig(mode='spectroscopy', filter='J', mask='my_MOS_mask')
mosfire_H = MOSFIREConfig(mode='spectroscopy', filter='H', mask='my_MOS_mask')
mosfire_K = MOSFIREConfig(mode='spectroscopy', filter='K', mask='my_MOS_mask')
print(mosfire_Y)
print(mosfire_J)
print(mosfire_H)
print(mosfire_K)

In [ ]:
abba = ABBA(offset=1.5)
print(abba)

In [ ]:
abba

In [ ]:
# sequence
myseq_mosfire = Sequence([SequenceElement(pattern=abba, detconfig=mosfire_180s, instconfig=mosfire_Y, repeat=5),
                          SequenceElement(pattern=abba, detconfig=mosfire_120s, instconfig=mosfire_J, repeat=8),
                          SequenceElement(pattern=abba, detconfig=mosfire_120s, instconfig=mosfire_H, repeat=8),
                          SequenceElement(pattern=abba, detconfig=mosfire_180s, instconfig=mosfire_K, repeat=5),
                         ])
print(myseq_mosfire)

## MOSFIRE Example 2

In [ ]:
singleobj_Y = MOSFIREConfig(mode='spectroscopy', filter='Y', mask='longslit_3x0.7')
singleobj_J = MOSFIREConfig(mode='spectroscopy', filter='J', mask='longslit_3x0.7')
singleobj_H = MOSFIREConfig(mode='spectroscopy', filter='H', mask='long2pos')
singleobj_K = MOSFIREConfig(mode='spectroscopy', filter='K', mask='long2pos')
print(mosfire_Y)
print(mosfire_J)
print(mosfire_H)
print(mosfire_K)

In [ ]:
long2pos = Long2pos()
print(long2pos)

In [ ]:
long2pos

In [ ]:
# sequence
myseq2_mosfire = Sequence([SequenceElement(pattern=abba, detconfig=mosfire_180s, instconfig=singleobj_Y, repeat=2),
                           SequenceElement(pattern=abba, detconfig=mosfire_120s, instconfig=singleobj_J, repeat=2),
                           SequenceElement(pattern=long2pos, detconfig=mosfire_120s, instconfig=singleobj_H, repeat=2),
                           SequenceElement(pattern=long2pos, detconfig=mosfire_180s, instconfig=singleobj_K, repeat=2),
                          ])
print(myseq2_mosfire)